In [1]:
import json
import pandas as pd
from tqdm import tqdm
import glob
import sqlite3


In [2]:
# def get_metadata(wiki_id):
#     query = """SELECT  ?subject ?subjectLabel
#         ?instance ?instanceLabel
#         ?subclass ?subclassLabel
#         ?inception
#         ?time_period ?time_periodLabel
#         ?culture ?cultureLabel 
#         ?architecture_style ?architecture_styleLabel 
#         ?founded_by ?founded_byLabel 
#         ?creator ?creatorLabel
#         ?country ?countryLabel
#         ?territory ?territoryLabel
#         ?genre ?genreLabel
#         ?movement ?movementLabel
#         ?author ?authorLabel
#         ?publicationdate
#         ?country_origin ?country_originLabel
#         ?wikipediaLink
# WHERE {
#   BIND(wd:%s AS ?subject)
#   OPTIONAL { ?subject wdt:P31 ?instance. }
#   OPTIONAL { ?subject wdt:P50 ?author. }
#   OPTIONAL { ?subject wdt:P279 ?subclass. }
#   OPTIONAL { ?subject wdt:P571 ?inception. }
#   OPTIONAL { ?subject wdt:P2348 ?time_period. }
#   OPTIONAL { ?subject wdt:P2596 ?culture. }
#   OPTIONAL { ?subject wdt:P112 ?founded_by. }
#   OPTIONAL { ?subject wdt:P170 ?creator. }
#   OPTIONAL { ?subject wdt:P17 ?country. }
#   OPTIONAL { ?subject wdt:P131 ?territory. }
#   OPTIONAL { ?subject wdt:P136 ?genre. }
#   OPTIONAL { ?subject wdt:P135 ?movement. }
#   OPTIONAL { ?subject wdt:P577 ?publicationdate. }
#   OPTIONAL { ?subject wdt:P495 ?country_origin. }
  
#   OPTIONAL {
#     ?wikipediaLink schema:about ?subject;
#                   schema:isPartOf <https://en.wikipedia.org/>.
#   }
  
#   SERVICE wikibase:label {
#     bd:serviceParam wikibase:language "en".
#   }
# }

In [3]:
list_df = []
list_df_instance = []
list_df_inception = []
list_df_subclass = []
list_df_genre = []
list_df_country = []
list_author = []
list_df_wikipedia = []
list_df_time_period = []
list_df_culture = []
list_df_publication_date = []
list_df_creator = []

paths = glob.glob('raw_data/extracts/*')

In [4]:
for path in tqdm(paths):

    with open(path, "r") as file:
        data = json.load(file)

    final = []
    for x in data:
        extracted_values = [{key: value['value'] for key, value in item.items()} for item in x]
        final.append(extracted_values)

    final_list=[]
    for x in final:
        res = pd.DataFrame(x)
        final_list.append(res)

    df = pd.concat([x for x in final_list])
    df['subject'] = df['subject'].apply(lambda x: x.split('http://www.wikidata.org/entity/')[1] if isinstance(x, str) and 'entity' in x else None)

    df_label = df[[ 'subject', 'subjectLabel']].drop_duplicates().dropna().reset_index(drop=True)
    list_df.append(df_label)

    try:
        df_instance = df[['subject', 'subjectLabel', 'instance', 'instanceLabel']].drop_duplicates().dropna().reset_index(drop=True)
        df_instance['instance'] = df_instance['instance'].apply(lambda x: x.split('http://www.wikidata.org/entity/')[1] if isinstance(x, str) and 'entity' in x else None)
        list_df_instance.append(df_instance)
    except:
        pass

    try:
        df_inception  = df[['subject', 'subjectLabel', 'inception']].drop_duplicates().dropna().reset_index(drop=True)
        list_df_inception.append(df_inception)
    except:
        pass

    try:
        df_subclass = df[['subject', 'subjectLabel', 'subclass', 'subclassLabel']].drop_duplicates().dropna().reset_index(drop=True)
        df_subclass['subclass'] = df_subclass['subclass'].apply(lambda x: x.split('http://www.wikidata.org/entity/')[1] if isinstance(x, str) and 'entity' in x else None)
        list_df_subclass.append(df_subclass)
    except:
        pass

    try:
        df_genre = df[['subject', 'subjectLabel', 'genre', 'genreLabel']].drop_duplicates().dropna().reset_index(drop=True)
        df_genre['genre'] = df_genre['genre'].apply(lambda x: x.split('http://www.wikidata.org/entity/')[1] if isinstance(x, str) and 'entity' in x else None)
        list_df_genre.append(df_genre)
    except:
        pass

    try:
        df_country= df[['subject', 'subjectLabel', 'country_origin', 'country_originLabel']].drop_duplicates().dropna().reset_index(drop=True)
        df_country['country_origin'] = df_country['country_origin'].apply(lambda x: x.split('http://www.wikidata.org/entity/')[1] if isinstance(x, str) and 'entity' in x else None)
        list_df_country.append(df_country)
    except:
        pass

    try:
        df_author= df[['subject', 'subjectLabel', 'author', 'authorLabel']].drop_duplicates().dropna().reset_index(drop=True)
        df_author['author'] = df_author['author'].apply(lambda x: x.split('http://www.wikidata.org/entity/')[1] if isinstance(x, str) and 'entity' in x else None)
        list_author.append(df_author)
    except:
        pass

    try:
        df_wiki= df[['subject', 'subjectLabel','wikipediaLink']].drop_duplicates().dropna().reset_index(drop=True)
        list_df_wikipedia.append(df_wiki)
    except:
        pass

    try:
        df_time_period= df[['subject', 'subjectLabel', 'time_period', 'time_periodLabel']].drop_duplicates().dropna().reset_index(drop=True)
        df_time_period['time_period'] = df_time_period['time_period'].apply(lambda x: x.split('http://www.wikidata.org/entity/')[1] if isinstance(x, str) and 'entity' in x else None)
        list_df_time_period.append(df_time_period)
    except:
        pass

    try:
        df_culture= df[['subject', 'subjectLabel', 'culture', 'cultureLabel']].drop_duplicates().dropna().reset_index(drop=True)
        df_culture['culture'] = df_culture['culture'].apply(lambda x: x.split('http://www.wikidata.org/entity/')[1] if isinstance(x, str) and 'entity' in x else None)
        list_df_culture.append(df_culture)
    except:
        pass

    try:
        df_publication_date= df[['subject', 'subjectLabel', 'publicationdate']].drop_duplicates().dropna().reset_index(drop=True)
        list_df_publication_date.append(df_publication_date)
    except:
        pass

    try:
        df_creator= df[['subject', 'subjectLabel', 'creator', 'creatorLabel']].drop_duplicates().dropna().reset_index(drop=True)
        df_creator['creator'] = df_creator['creator'].apply(lambda x: x.split('http://www.wikidata.org/entity/')[1] if isinstance(x, str) and 'entity' in x else None)
        list_df_creator.append(df_creator)
    except:
        pass

  0%|          | 0/25 [00:00<?, ?it/s]

100%|██████████| 25/25 [01:29<00:00,  3.58s/it]


In [5]:
# Connect to the SQLite database
conn = sqlite3.connect('literay-works.db')

In [6]:
final_df = pd.concat([x for x in list_df])
final_df = final_df.reset_index(drop=True)
final_df.to_sql('objects', conn, if_exists='replace', index=False)

final_df_instance = pd.concat([x for x in list_df_instance])
final_df_instance = final_df_instance.reset_index(drop=True)
final_df_instance.to_sql('instance', conn, if_exists='replace', index=False)

final_df_inception = pd.concat([x for x in list_df_inception])
final_df_inception = final_df_inception.reset_index(drop=True)
final_df_inception.to_sql('inception', conn, if_exists='replace', index=False)

# final_df_subclass = pd.concat([x for x in list_df_subclass])
# final_df_subclass = final_df_subclass.reset_index(drop=True)
# final_df_subclass.to_sql('subclass', conn, if_exists='replace', index=False)

final_df_genre = pd.concat([x for x in list_df_genre])
final_df_genre = final_df_genre.reset_index(drop=True)
final_df_genre.to_sql('genre', conn, if_exists='replace', index=False)

final_df_wiki = pd.concat([x for x in list_df_wikipedia])
final_df_wiki = final_df_wiki.reset_index(drop=True)
final_df_wiki.to_sql('en_wikipedia_link', conn, if_exists='replace', index=False)

# final_df_time_period = pd.concat([x for x in list_df_time_period])
# final_df_time_period = final_df_time_period.reset_index(drop=True)
# final_df_time_period.to_sql('time_period', conn, if_exists='replace', index=False)

# final_df_culture = pd.concat([x for x in list_df_culture])
# final_df_culture = final_df_culture.reset_index(drop=True)
# final_df_culture.to_sql('culture', conn, if_exists='replace', index=False)

final_df_publication_date = pd.concat([x for x in list_df_publication_date])
final_df_publication_date = final_df_publication_date.reset_index(drop=True)
final_df_publication_date.to_sql('publication_date', conn, if_exists='replace', index=False)

# final_df_creator = pd.concat([x for x in list_df_creator])
# final_df_creator = final_df_creator.reset_index(drop=True)
# final_df_creator.to_sql('creator', conn, if_exists='replace', index=False)


final_df_country = pd.concat([x for x in list_df_country])
final_df_country = final_df_country.reset_index(drop=True)
final_df_country.to_sql('country', conn, if_exists='replace', index=False)

final_df_author = pd.concat([x for x in list_author])
final_df_author = final_df_author.reset_index(drop=True)
final_df_author.to_sql('author', conn, if_exists='replace', index=False)

204064